In [1]:
import pandas as pd

# classification dataset for voting ensemble
classification_data = pd.read_csv("./classification_data_cleaned.csv")
# regression dataset for decision tree models 
regression_data = pd.read_csv("./regression_data_cleaned_dead.csv")

